In [1]:
# coding: utf-8
# Module ?: 

"""
Deve ler o arquivo de dados de mercado, caso exista, ou criá-lo a partir de dados da corretora, respeitando os 
parâmetros do arquivo de parâmetros.
"""

import sys
import os

sys.path.append(os.getcwd().split(sep="notebooks")[0]) # Trocar o noteboooks antes de passar pro .py
#import modules
from modules.db_data import db_feeder as db
from elasticsearch import Elasticsearch

from elasticsearch_dsl import Document, Float
from elasticsearch_dsl.connections import connections

import pandas as pd

from datetime import datetime
import time

In [2]:
symbol = 'BTCUSDT'
interval = '1m'
start_str = '1 hour ago UTC'

In [3]:
from binance.client import Client
client = Client("","")
es = Elasticsearch(hosts=['localhost:9200'])

In [176]:
es.indices.delete(index='marketdata-btcusdt-binance', ignore=[400, 404])

{'acknowledged': True}

In [132]:
hist = client.get_historical_klines(symbol,interval,start_str)
del hist[-1]
print(datetime.utcfromtimestamp(hist[len(hist)-1][0]/1000))

2019-02-25 13:21:00


In [103]:
#es.indices.delete(index='marketdata-btcusdt-binance', ignore=[400, 404])

In [147]:
connections.create_connection(hosts=['localhost:9200'])

class Candle(Document):
    Open = Float()
    High = Float()
    Low = Float()
    Close = Float()
    Volume = Float()
    Close_time = Float()
    Quote_asset_vol = Float()
    N_of_trades = Float()
    Taker_buy_base_asset_vol = Float()
    Taker_buy_quote_asset_vol = Float()
    
    class Index:
        name = 'marketdata-btcusdt-binance'
    
    def save(self, **kwargs):
        return super(Candle, self).save(**kwargs)
    
def indexing_new_data(hist):
    """ Indexes list 'hist' received by binance.
    """
    for i in range(0,len(hist)): #Indexing 'Hist'
        candle = Candle(meta={'id': hist[i][0]})
        candle.Open = float(hist[i][1])
        candle.High = float(hist[i][2])
        candle.Low = float(hist[i][3])
        candle.Close = float(hist[i][4])
        candle.Volume = float(hist[i][5])
        candle.Close_time = hist[i][6]
        candle.Quote_asset_vol = float(hist[i][7])
        candle.N_of_trades = float(hist[i][8])
        candle.Taker_buy_base_asset_vol = float(hist[i][9])
        candle.Taker_buy_quote_asset_vol = float(hist[i][10])
    
        candle.save()
    

  


Dados atualizados: 
 Ultimo Candle coletado: 2019-02-27 13:43:00


In [170]:
try:
    res=es.search(index='marketdata-btcusdt-binance', size = 1, sort = "_id:desc") # Indentify last ID: Open_time
    close_time = int(res['hits']['hits'][0]['_source']['Close_time'])
    hist_new = db.get_historical_klines_from_ts(symbol,interval,close_time)
    del hist_new[-1]
    if len(hist_new):
        Candle.init()
        indexing_new_data(hist_new)
        time.sleep(1)
        res2=es.search(index='marketdata-btcusdt-binance', size = 1, sort = "_id:desc")
        u_c = datetime.utcfromtimestamp(float(res2['hits']['hits'][0]['_id'])/1000)
        print('Dados atualizados: \n Ultimo Candle coletado:',u_c)
    else:
        print('Tempo curto. Nao foi possivel atualizar candle.')
except: # In case index doens't exist
    Candle.init()
    indexing_new_data(hist)
    print('Index criado')

Dados atualizados: 
 Ultimo Candle coletado: 2019-02-27 13:57:00


In [171]:
dt = float(res2['hits']['hits'][0]['_id'])/1000 - time.time()+120
dt

42.73247408866882

In [164]:
close_time = float(res['hits']['hits'][0]['_source']['Close_time'])

In [24]:
res

{'took': 250,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 60,
  'max_score': None,
  'hits': [{'_index': 'marketdata-btcusdt-binance',
    '_type': 'doc',
    '_id': '1550750040000',
    '_score': None,
    '_source': {'Open': 3922.5,
     'High': 3923.06,
     'Low': 3921.8,
     'Close': 3922.99,
     'Volume': 3.276268,
     'Close_time': 1550750099999.0,
     'Quote_asset_vol': 12851.73789994,
     'N_of_trades': 47.0,
     'Taker_buy_base_asset_vol': 2.126405,
     'Taker_buy_quote_asset_vol': 8341.32132994},
    'sort': ['1550750040000']}]}}

In [118]:
hist_new = db.get_historical_klines_from_ts(symbol,interval,int(close_time))

In [119]:
hist_new[0]

[1551099060000,
 '3800.59000000',
 '3802.19000000',
 '3798.00000000',
 '3801.96000000',
 '40.87638800',
 1551099119999,
 '155348.38237033',
 160,
 '27.52299500',
 '104601.48427089',
 '0']

In [120]:
hist[len(hist)-1]

[1551099000000,
 '3803.74000000',
 '3803.96000000',
 '3798.61000000',
 '3800.02000000',
 '18.87867300',
 1551099059999,
 '71768.83035888',
 149,
 '9.49214500',
 '36087.01359628',
 '0']

In [174]:
Candle.Index.name = 'teste'

In [175]:
Candle.Index.name

'teste'